In [1]:
!pwd

/Users/mkeller/research/dbmi/vitessce/paper-figures/osmfish/src


In [2]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    DataType as dt,
    FileType as ft,
    AnnDataWrapper,
    OmeZarrWrapper,
    BASE_URL_PLACEHOLDER
)
from os.path import join
import json

In [3]:
cells_zarr = "codeluppi_2018_nature_methods.cells.h5ad.zarr"
molecules_zarr = "codeluppi_2018_nature_methods.molecules.h5ad.zarr"
image_zarr = "codeluppi_2018_nature_methods.image.ome.zarr"

In [5]:
BASE_DIR = join("..", "data", "processed")

In [31]:
vc = VitessceConfig(schema_version="1.0.15", name="Codeluppi et al.", description="osmFISH", base_dir=BASE_DIR)
dataset = vc.add_dataset(name="osmFISH").add_object(AnnDataWrapper(
    adata_path=cells_zarr,
    obs_feature_matrix_path="layers/X_uint8",
    obs_segmentations_path="obsm/X_segmentations",
    obs_locations_path="obsm/X_spatial",
    obs_embedding_paths=["obsm/X_pca", "obsm/X_tsne"],
    obs_embedding_names=["PCA", "t-SNE"],
    obs_set_paths=[["obs/Cluster", "obs/Subcluster"]],
    obs_set_names=["Cell Type"],
)).add_object(
    OmeZarrWrapper(img_path=image_zarr),
)
""".add_object(AnnDataWrapper(
    adata_path=molecules_zarr,
    obs_locations_path="obsm/X_spatial",
    obs_labels_path=["obs/Gene"],
    coordination_values={
        "obsType": "molecule"
    }
))
"""

#status = vc.add_view(vt.STATUS, dataset=dataset, x=0, y=5, w=2, h=1)
#description = vc.add_view(vt.DESCRIPTION, dataset=dataset, x=0, y=0, w=2, h=1)
#heatmap = vc.add_view(vt.HEATMAP, dataset=dataset, x=2, y=4, w=5, h=2).set_props(transpose=True)

lc = vc.add_view(vt.LAYER_CONTROLLER, dataset=dataset, x=0, y=0, w=3, h=4)
feature_list = vc.add_view(vt.FEATURE_LIST, dataset=dataset, x=0, y=4, w=3, h=4)
obs_sets = vc.add_view(vt.OBS_SETS, dataset=dataset, x=0, y=8, w=3, h=4)


spatial_cell_sets = vc.add_view(vt.SPATIAL, dataset=dataset, x=3, y=0, w=3, h=6)
spatial_lamp5 = vc.add_view(vt.SPATIAL, dataset=dataset, x=6, y=0, w=3, h=6)
spatial_rorb = vc.add_view(vt.SPATIAL, dataset=dataset, x=9, y=0, w=3, h=6)


tsne = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="t-SNE", x=3, y=6, w=3, h=6)

violin_plot_lamp5 = vc.add_view(vt.OBS_SET_FEATURE_VALUE_DISTRIBUTION, dataset=dataset, x=6, y=6, w=3, h=6)
violin_plot_rorb = vc.add_view(vt.OBS_SET_FEATURE_VALUE_DISTRIBUTION, dataset=dataset, x=9, y=6, w=3, h=6)

all_views = [
    lc,
    feature_list,
    obs_sets,
    
    spatial_cell_sets,
    spatial_lamp5,
    spatial_rorb,
    
    tsne,
    violin_plot_lamp5,
    violin_plot_rorb,
]

image_layer = [{
    "type": "raster",
    "index": 0,
    "visible": True,
    "colormap": None,
    "opacity": 1,
  "domainType": "Min/Max",
  "transparentColor": None,
  "renderingMode": "Additive",
  "use3d": False,
  "channels": [
    {
      "selection": {
        "c": 1
      },
      "color": [
        255,
        255,
        255
      ],
      "visible": True,
      "slider": [
        1,
        1473
      ]
    },
    {
      "selection": {
        "c": 0
      },
      "color": [
        255,
        255,
        255
      ],
      "visible": True,
      "slider": [
        1,
        447
      ]
    }
  ]
}]
segmentation_layer = {
    "opacity": 1,
    "radius": 0,
    "visible": True,
    "stroked": False,
}
point_layer = {
    "opacity": 1,
    "radius": 20,
    "visible": True,
}

spatial_views = [
    lc,
    
    spatial_cell_sets,
    spatial_lamp5,
    spatial_rorb,
]

vc.link_views(spatial_views, [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y, ct.SPATIAL_IMAGE_LAYER, ct.SPATIAL_SEGMENTATION_LAYER], [-5.5, 16000, 20000, image_layer, segmentation_layer])
vc.link_views([tsne], [ct.EMBEDDING_ZOOM], [0.75])

cell_set_selection = [
    ["Cell Type", "Excitatory neurons", "Pyramidal L2-3"],
    ["Cell Type", "Excitatory neurons", "Pyramidal L2-3 L5"],
    ["Cell Type", "Excitatory neurons", "Pyramidal L3-4"],
    ["Cell Type", "Excitatory neurons", "pyramidal L4"],
]
vc.link_views([tsne, obs_sets, spatial_cell_sets, violin_plot_lamp5, violin_plot_rorb], [ct.OBS_SET_SELECTION], [cell_set_selection])
vc.link_views([spatial_cell_sets, tsne], [ct.OBS_COLOR_ENCODING], ["cellSetSelection"])

vc.link_views([violin_plot_lamp5, spatial_lamp5], [ct.FEATURE_SELECTION], [["Lamp5"]])
vc.link_views([violin_plot_rorb, spatial_rorb, feature_list], [ct.FEATURE_SELECTION], [["Rorb"]])


In [32]:
#vw = vc.widget(height=800)
#vw

In [33]:
vc.web_app()

'http://vitessce.io/#?theme=light&url=data:,%7B%22version%22%3A+%221.0.15%22%2C+%22name%22%3A+%22Codeluppi+et+al.%22%2C+%22description%22%3A+%22osmFISH%22%2C+%22datasets%22%3A+%5B%7B%22uid%22%3A+%22A%22%2C+%22name%22%3A+%22osmFISH%22%2C+%22files%22%3A+%5B%7B%22fileType%22%3A+%22anndata.zarr%22%2C+%22url%22%3A+%22http%3A%2F%2Flocalhost%3A8008%2Fcodeluppi_2018_nature_methods.cells.h5ad.zarr%22%2C+%22options%22%3A+%7B%22obsLocations%22%3A+%7B%22path%22%3A+%22obsm%2FX_spatial%22%7D%2C+%22obsSegmentations%22%3A+%7B%22path%22%3A+%22obsm%2FX_segmentations%22%7D%2C+%22obsEmbedding%22%3A+%5B%7B%22path%22%3A+%22obsm%2FX_pca%22%2C+%22dims%22%3A+%5B0%2C+1%5D%2C+%22embeddingType%22%3A+%22PCA%22%7D%2C+%7B%22path%22%3A+%22obsm%2FX_tsne%22%2C+%22dims%22%3A+%5B0%2C+1%5D%2C+%22embeddingType%22%3A+%22t-SNE%22%7D%5D%2C+%22obsSets%22%3A+%5B%7B%22name%22%3A+%22Cell+Type%22%2C+%22path%22%3A+%5B%22obs%2FCluster%22%2C+%22obs%2FSubcluster%22%5D%7D%5D%2C+%22obsFeatureMatrix%22%3A+%7B%22path%22%3A+%22layers%2FX_u

In [9]:
config_dict = vc.to_dict(base_url=BASE_URL_PLACEHOLDER)

# Use `open` to create a new empty file at ./exported_data/vitessce.json
with open(join(BASE_DIR, "osmfish.json"), "w") as f:
    json.dump(config_dict, f)
